# Exercice 2

## Heuristiques

Les Heuristiques choisies pour effectuer le matching sont un algorithme **Greedy** et l'algorithme **KarpSipser**

### Setup

In [8]:
import networkx as nx
import debug_printers as deb

# Defines whether the debug printers actually do stuff.
deb.gb_doPrintDebug = True

def gen_graph(n: int = 120, p: float = 0.04) -> (nx.Graph, dict):
    r"""Returns an Erdös-Renyi Graph with the given parameters.

    Parameters:
    -----------
    n : int
        The number of nodes (120 by default).
    p : float
        The probability for edge creation (0.04 by default).

    Returns:
    --------
    A newly created Erdös-Renyi Graph and it's associated layout.
    """
    G = nx.gnp_random_graph(n, p)
    pos = nx.circular_layout(G)
    deb.draw(G, prefix="E2_Matching_", pos=pos)
    return (G, pos)

## Algorithme Greedy

### Implémentation

In [9]:
import networkx as nx

def greedy(G: nx.Graph) -> set:
    r"""Uses a Greedy algorithm to create a matching seet within the Graph

    Parameters:
    -----------
    G : networkx.Graph
        The graph to search a matching in.

    Returns:
    --------
    A matching set.
    """

    # Preserve the original graph
    graph: nx.Graph = G.copy()

    matching = set()

    while graph.order() > 0:

        # Pick node
        for node in graph.nodes:
            cur_node = node
            node_edges = list(graph.edges(cur_node))

            # If there are no edges, skip rest
            if len(node_edges) == 0:
                break

            matching_edge = node_edges[0]

            # If it is of degree one, pick it
            if graph.degree(node) == 1:
                break
            # If not, pick the node with the least edges
            else:
                for n in node_edges[0]:
                    if n == node:
                        continue
                    other_node = n
                for edge in node_edges:
                    for node_edge in edge:
                        if node == node_edge:
                            continue
                        if len(graph.edges(node_edge)) < len(graph.edges(other_node)):
                            other_node = node_edge
                            matching_edge = edge

        # If there are no edges, remove the node
        if len(node_edges) == 0:
            graph.remove_node(cur_node)
            continue

        # Add node to the matching set
        matching.add(matching_edge)
        # Remove nodes associated with the edge
        for node in matching_edge:
            graph.remove_node(node)

    return matching

deb.gb_doPrintDebug = False
greedy(gen_graph()[0])

{(2, 73),
 (3, 102),
 (5, 70),
 (6, 82),
 (8, 43),
 (9, 14),
 (11, 4),
 (15, 101),
 (17, 46),
 (19, 21),
 (26, 24),
 (28, 76),
 (32, 12),
 (33, 85),
 (36, 25),
 (45, 35),
 (47, 41),
 (48, 29),
 (49, 39),
 (51, 66),
 (53, 67),
 (55, 90),
 (58, 27),
 (60, 31),
 (69, 38),
 (71, 59),
 (77, 95),
 (78, 37),
 (79, 1),
 (81, 63),
 (83, 68),
 (84, 99),
 (87, 0),
 (88, 18),
 (89, 52),
 (91, 44),
 (92, 50),
 (93, 34),
 (94, 75),
 (96, 10),
 (97, 57),
 (98, 16),
 (100, 56),
 (103, 40),
 (104, 7),
 (105, 80),
 (106, 22),
 (107, 61),
 (108, 62),
 (109, 42),
 (110, 20),
 (112, 74),
 (113, 23),
 (114, 111),
 (115, 30),
 (116, 64),
 (117, 72),
 (118, 65),
 (119, 13)}

### Complexité

L'algorithme **Greedy** passe par chacun des sommets du graphe afin de déterminer lesquels peuvent être matchés en choisissant ceux qui ont le moins d'arrêtes et répète ce processus jusqu'à l'épuisement de nodes dans le graphe. De ce fait, la complexité de l'algorithme est `O(n²)`. En effet, dans le cas où chaque sommet est connecté à tous les autres sommets, tous seronts testés pour chaques sommets, soit `n * n` avec `n` le nombre de sommets.
L'algorithme tourne donc en temps polynomial.

## KarpSipser

### Implémentation

In [10]:
import networkx as nx

def karp_sipser(G: nx.Graph) -> set:
    r"""Uses KarpSipser's algorithm to create a matching set within the Graph

    Parameters:
    -----------
    G : networkx.Graph
        The graph to search a matching in.

    Returns:
    --------
    A matching set.
    """

    # Preserve the original graph
    graph: nx.Graph = G.copy()

    matching = set()

    while graph.order() > 0:

        # Pick node
        for node in graph.nodes:
            # if it is of degree one, pick it
            if graph.degree(node) == 1:
                cur_node = node
                break
        # If no node are found, take the first one
        else:
            cur_node = list(graph.nodes)[0]

        node_edges = list(graph.edges(cur_node))

        # If there are no edges, remove the node
        if len(node_edges) == 0:
            graph.remove_node(cur_node)
            continue

        # Add node to the matching set
        edge = node_edges[0]
        matching.add(edge)
        # Remove nodes associated with the edge
        for node in edge:
            graph.remove_node(node)

    return matching

### Complexité

L'algorithme **KarpSipser** passe par chacun des sommets du graphe afin de déterminer lesquels ne possèdent qu'une seule arrête et répète ce processus jusqu'à l'épuisement de nodes dans le graphe. De ce fait, la complexité de l'algorithme est `O(n²)`. En effet, dans le cas où aucun sommet ne possède d'arrêtes, tous les sommets seronts testés pour chaques sommets, soit `n * n` avec `n` le nombre de sommets.
L'algorithme tourne donc en temps polynomial.

## Comparaison

Comparaison des algorithmes **Greedy** et **KarpSipser**. La méthode de mathing de la librairie `networkx` est aussi testée en tant que référence.

In [11]:
import debug_printers as deb
from save_and_load import load_from_edgelist, save_as_edgelist
from test_matching import test_matching
from os import path, mkdir, linesep
import time

# Defines whether the debug printers actually do stuff.
deb.gb_doPrintDebug = False

max_tries: int = 1000

dirname = f"{path.curdir}/ex02_graphs"

# Generate max_tries graphs for legit comparison
# Set to True to generate new graphs
if False:
    if not path.exists(dirname):
        mkdir(dirname)

    for i in range(0, max_tries):
        save_as_edgelist(gen_graph()[0], f"{dirname}/graph{i}.txt")

values = [[]]
for i in range(0, max_tries):
    values.append([])

def attempt_matching(fn):
    tottime = 0
    for i in range(0, max_tries):
        G = load_from_edgelist(f"{dirname}/graph{i}.txt")[0]
        stime = time.time()
        match: set = fn(G)
        ttime = time.time() - stime
        tottime = ttime + tottime
        if not test_matching(G, match):
            print(f"{i:03d} : matching failed.")
            values[i].append((-1, -1))
        else:
            values[i].append((len(match), ttime))
    return tottime

# Bench-marking thanks to networkx's maximal_matching.
print("Running builtin")
print(f"Completed {max_tries} in {attempt_matching(nx.maximal_matching)}")

# Testing Greedy algorithm.
print(f"{linesep}Running Greedy")
print(f"Completed {max_tries} in {attempt_matching(greedy)}")

# Testing KarpSipser algorithm.
print(f"{linesep}Running KarpSipser")
print(f"Completed {max_tries} in {attempt_matching(karp_sipser)}")

print(f"{linesep}    |          builtin         |          Greedy          |        KarpSipser")
print("idx | matches | time (seconds) | matches | time (seconds) | matches | time (seconds)")
averages = [float(0.0), float(0.0), float(0.0), float(0.0), float(0.0), float(0.0)]
for i in range(0, max_tries):
    print(f"{i:03d} | {values[i][0][0]: 7d} | {values[i][0][1]: 14f} | {values[i][1][0]: 7d} | {values[i][1][1]: 14f} | {values[i][2][0]: 7d} | {values[i][2][1]: 14f}")
    if values[i][0][0] != -1:
        averages[0] = (averages[0] + values[i][0][0]) / 2
        averages[1] = (averages[1] + values[i][0][1]) / 2
    if values[i][1][0] != -1:
        averages[2] = (averages[2] + values[i][1][0]) / 2
        averages[3] = (averages[3] + values[i][1][1]) / 2
    if values[i][2][0] != -1:
        averages[4] = (averages[4] + values[i][2][0]) / 2
        averages[5] = (averages[5] + values[i][2][1]) / 2

print("    | average |  average time  | average |  average time  | average |  average time")
print(f"    | {averages[0]: 2.4f} | {averages[1]: 14f} | {averages[2]: 2.4f} | {averages[3]: 14f} | {averages[4]: 2.4f} | {averages[5]: 14f}")

Running builtin
Completed 1000 in 0.1801927089691162

Running Greedy
Completed 1000 in 152.7820942401886

Running KarpSipser
Completed 1000 in 3.0245349407196045

    |          builtin         |          Greedy          |        KarpSipser
idx | matches | time (seconds) | matches | time (seconds) | matches | time (seconds)
000 |      52 |       0.000393 |      59 |       0.148907 |      59 |       0.004160
001 |      51 |       0.000378 |      59 |       0.121004 |      59 |       0.002688
002 |      53 |       0.000343 |      59 |       0.116471 |      56 |       0.003438
003 |      51 |       0.000222 |      60 |       0.145947 |      59 |       0.002854
004 |      50 |       0.000265 |      59 |       0.132076 |      58 |       0.002989
005 |      50 |       0.000230 |      58 |       0.137661 |      58 |       0.003581
006 |      49 |       0.000531 |      60 |       0.124079 |      59 |       0.003398
007 |      49 |       0.000217 |      59 |       0.131520 |      59 |       0.0

De par les tests de performances effectués, il s'avère que l'algorithme **Greedy** est bien moins performant que l'algorithme **KarpSipser**. De plus, il semble que le nombre de set trouvés est similaire entre les 2 algoritmes.